In [1]:
from langame import LangameClient
from pprint import pprint
from firebase_admin import firestore
import requests
from bs4 import BeautifulSoup
import json
import openai
from pathlib import Path
import pandas as pd
import datetime
import markdown
c = LangameClient(path_to_config_file="../config.yaml")

In [2]:
!ls ../data/

augment
ice_breaker_2021_11_28_dedup_2021_12_03.txt
ice_breaker_2021_11_28.jsonl
ice_breaker_2021_11_28.txt
ice_breaker_2021_11_29.txt
ice_breaker_2021_12_03.txt
ice_breaker_26_11_2021.jsonl
openai_fine_tune_ice_breaker_26_11_2021.jsonl
openai_fine_tune_science_26_11_2021.jsonl
science_2021_11_28_dedup_2021_12_03.txt
science_2021_11_28.txt
science_26_11_2021.jsonl


In [3]:
out_file_name = "../data/fine_tune_ice_breaker_2021_12_03.jsonl"
in_file_name = "../data/ice_breaker_2021_12_03.txt"

with open(in_file_name) as f:
  for e in f.readlines():
    splitted = [j.strip() for j in e.split("###")]
    if len(splitted) != 2:
      continue
    topics, starter = splitted
    if not topics or not starter:
      continue
    with open(out_file_name, "a+") as outfile:
      json.dump({
          "prompt": f"",
          "completion": f" {topics} ### {starter}\n",
      }, outfile)
      outfile.write('\n')

In [4]:
!head -n 10 $out_file_name

{"prompt": "", "completion": " ice breaker ### What do you wish everyone knew?\n"}
{"prompt": "", "completion": " ice breaker ### What's something you should thank your parents for?\n"}
{"prompt": "", "completion": " art,ice breaker ### If you could take on a new persona what would it be?\n"}
{"prompt": "", "completion": " ice breaker ### Where is the most interesting andWhy?\n"}
{"prompt": "", "completion": " ice breaker ### What is the biggest risk you have ever taken that turned out to be a good thing?\n"}
{"prompt": "", "completion": " philosophy,ice breaker ### What books on philosophy have had the largest influence on you?\n"}
{"prompt": "", "completion": " ice breaker ### What was one of your proudest moments?\n"}
{"prompt": "", "completion": " ice breaker ### If given a choice, would you prefer to recall a memory from early childhood or your last day on earth?\n"}
{"prompt": "", "completion": " ice breaker ### What is something people do that you do not understand?\n"}
{"prompt

In [ ]:
!openai tools fine_tunes.prepare_data -f $out_file_name

In [6]:
f = openai.File.create(
  file=open(out_file_name),
  purpose="fine-tune"
)
f_t = openai.FineTune.create(
    training_file=f["id"],
    model="curie"
)

In [7]:
model = openai.FineTune.list()['data'][-1]
model

<FineTune fine-tune id=ft-yKLdmtM2Zluct0H2uxIZnC56 at 0x7fa03209a770> JSON: {
  "created_at": 1638707165,
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": null,
    "n_epochs": 4,
    "prompt_loss_weight": 0.1,
    "use_packing": null
  },
  "id": "ft-yKLdmtM2Zluct0H2uxIZnC56",
  "model": "curie",
  "object": "fine-tune",
  "organization_id": "org-KwcHNgfGe4pqdKDLQIJt99UZ",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 9435397,
      "created_at": 1638707165,
      "filename": "fine_tune_ice_breaker_2021_12_03.jsonl",
      "id": "file-TTOJYXIxaDO2eVRJUHaCTNN8",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "status_details": null
    }
  ],
  "updated_at": 1638707166,
  "validation_files": []
}

In [34]:
openai.Completion.create(
  model=model["fine_tuned_model"],
  prompt="",
  max_tokens=300,
  stop=["\n###\n", "###"],
)

<OpenAIObject text_completion id=cmpl-3bb9PbAkCra9NSEJrOTKDhDxi4yCP at 0x13f15c770> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "While most of humanity is focused on avoiding harm, a small but very vocal minority has decided that the best way to avoid harm is to actively seeks it out. Called \u201cthe optimists\u201d by the media, this group includes anarchists, Daredevil bloggers, thrill-seekers, and other \u2013 sometimes bizarre \u2013 personalities."
    }
  ],
  "created": 1630089955,
  "id": "cmpl-3bb9PbAkCra9NSEJrOTKDhDxi4yCP",
  "model": "curie:ft-louis-personal-2021-08-27-16-57-14",
  "object": "text_completion"
}